# SVM para test luminaria


In [26]:
import pandas as pd
import numpy as np

from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [27]:
df=pd.read_csv('datasets/luminaria_con_niveles_de_riesgo 231.csv')
df.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord,nivel_riesgo
0,7mo o más,No,Sí,No,"Sí, medio tiempo",Algo,No,Algunas veces,Insatisfecho/a,Algunas veces,Alto Riesgo
1,7mo o más,No,Sí,Sí,"Sí, medio tiempo",Algo,No,Siempre,Insatisfecho/a,"Sí, frecuentemente",Alto Riesgo
2,1er - 3er,Si,Sí,Sí,No,Algo,No,Algunas veces,Insatisfecho/a,Nunca,Medio Riesgo
3,1er - 3er,Si,No,Sí,"Sí, medio tiempo",Algo,No,Algunas veces,Satisfecho/a,Algunas veces,Alto Riesgo
4,7mo o más,No,No,No,"Sí, medio tiempo",No afecta/No trabajo,No,Algunas veces,Satisfecho/a,"Sí, frecuentemente",Bajo Riesgo


In [28]:
# dividimos el data set en 3 partes
# entrenamiento 70%, validacion 15% y prueba 15%

from sklearn.model_selection import train_test_split
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['nivel_riesgo'],)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['nivel_riesgo'],)
print(f'Tamaño del conjunto de entrenamiento: {len(df_train)}')
print(f'Tamaño del conjunto de validación: {len(df_val)}')
print(f'Tamaño del conjunto de prueba: {len(df_test)}')

Tamaño del conjunto de entrenamiento: 161
Tamaño del conjunto de validación: 35
Tamaño del conjunto de prueba: 35


In [29]:
X_train = df_train.drop('nivel_riesgo', axis=1)
y_train = df_train['nivel_riesgo']

X_val = df_val.drop('nivel_riesgo', axis=1)
y_val = df_val['nivel_riesgo']

X_test = df_test.drop('nivel_riesgo', axis=1)
y_test = df_test['nivel_riesgo']

In [30]:
class SVMPreprocessor(BaseEstimator, TransformerMixin):
    """
    Encoder consistente para SVM según agent/encoding_svm_bayes.json.
    - No toca la columna 'nivel_riesgo'.
    - Produce columnas numéricas: features_numericas + one-hot (drop_first=True).
    - Mantiene orden y nombres de columnas fijos para producción.
    """
    def __init__(self):
        # Mapeos (incluye variantes comunes de acentos/formato)
        self.mappings = {
            'desmotivacion_bin': {'Sí':1, 'Si':1, 'Sí':1, 'No':0},
            'considerado_abandonar_bin': {'Sí':1, 'Si':1, 'No':0},
            'dificultades_economicas_bin': {'Sí':1, 'Si':1, 'No':0},
            'reprobo_materias_bin': {'Sí':1, 'Si':1, 'No':0},
            'semestre_ord': {'1°-3°':1, '1er - 3er':1, '4°-6°':2, '4to - 6to':2, '7° o más':3, '7mo o más':3},
            'impacto_laboral_ord': {'No afecta / No trabajo':0, 'No afecta/No trabajo':0, 'No afecta / No trabajo ':0,
                                   'Algo':1, 'Sí, mucho':2, 'Si, mucho':2},
            'apoyo_institucional_ord': {'Nunca':0, 'Algunas veces':1, 'Siempre':2},
            'satisfaccion_servicios_ord': {'Muy insatisfecho/a':0, 'Insatisfecho/a':1, 'Satisfecho/a':2, 'Muy satisfecho/a':3},
            'actividades_extracurriculares_ord': {'Nunca':0, 'Algunas veces':1, 'Sí, frecuentemente':2, 'Si, frecuentemente':2},
            'empleo_ord': {'No':0, 'Sí, medio tiempo':1, 'Si, medio tiempo':1, 'Sí, tiempo completo':2, 'Si, tiempo completo':2}
        }

        # categorías fijas para one-hot (seguro consistente en producción)
        self.empleo_cats = ["No", "Sí, medio tiempo", "Sí, tiempo completo"]
        self.impacto_cats = ["No afecta / No trabajo", "Algo", "Sí, mucho"]

        # Features finales (orden fijo)
        self.features_numericas = [
            "semestre_ord",
            "dificultades_economicas_bin",
            "reprobo_materias_bin",
            "apoyo_institucional_ord",
            "satisfaccion_servicios_ord",
            "actividades_extracurriculares_ord"
        ]
        # nombres one-hot según drop_first=True (se omite la primera categoría de referencia)
        self.features_onehot = [
            "empleo_Sí, medio tiempo",      # empleo_medio_tiempo
            "empleo_Sí, tiempo completo",   # empleo_tiempo_completo
            "impacto_Algo",
            "impacto_Sí, mucho"
        ]
        # columna objetivo derivada (opcional): 'riesgo_desercion'
        self.target_col = "riesgo_desercion"

    def fit(self, X: pd.DataFrame, y=None):
        # nothing to fit, but validate columns presence if possible
        return self

    def _map_column(self, s: pd.Series, mapping: dict):
        # si ya es numérico, devolver tal cual (preservar)
        if pd.api.types.is_numeric_dtype(s):
            return s.astype(float)
        mapped = s.map(mapping)
        # si quedan NaNs intentar convertir a numérico (valores ya codificados como strings)
        fallback = pd.to_numeric(s, errors='coerce')
        return mapped.combine_first(fallback).astype(float)

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()

        # aplicar mapeos para columnas que pueden venir en texto
        # manejar variantes: columnas originales o ya renombradas
        # MAPEOS BINARIOS/ORDINALES
        # desmotivacion_bin / desmotivacion
        if 'desmotivacion_bin' in df.columns:
            df['desmotivacion_bin'] = self._map_column(df['desmotivacion_bin'], self.mappings['desmotivacion_bin'])
        elif 'desmotivacion' in df.columns:
            df['desmotivacion_bin'] = self._map_column(df['desmotivacion'], self.mappings['desmotivacion_bin'])

        # considerado_abandonar
        if 'considerado_abandonar_bin' in df.columns:
            df['considerado_abandonar_bin'] = self._map_column(df['considerado_abandonar_bin'], self.mappings['considerado_abandonar_bin'])
        elif 'considerado_abandonar' in df.columns:
            df['considerado_abandonar_bin'] = self._map_column(df['considerado_abandonar'], self.mappings['considerado_abandonar_bin'])

        # dificultades_economicas
        if 'dificultades_economicas_bin' in df.columns:
            df['dificultades_economicas_bin'] = self._map_column(df['dificultades_economicas_bin'], self.mappings['dificultades_economicas_bin'])
        elif 'dificultades_economicas' in df.columns:
            df['dificultades_economicas_bin'] = self._map_column(df['dificultades_economicas'], self.mappings['dificultades_economicas_bin'])

        # reprobo_materias
        if 'reprobo_materias_bin' in df.columns:
            df['reprobo_materias_bin'] = self._map_column(df['reprobo_materias_bin'], self.mappings['reprobo_materias_bin'])
        elif 'reprobo_materias' in df.columns:
            df['reprobo_materias_bin'] = self._map_column(df['reprobo_materias'], self.mappings['reprobo_materias_bin'])

        # semestre
        if 'semestre_ord' in df.columns:
            df['semestre_ord'] = self._map_column(df['semestre_ord'], self.mappings['semestre_ord'])
        elif 'semestre' in df.columns:
            df['semestre_ord'] = self._map_column(df['semestre'], self.mappings['semestre_ord'])

        # apoyo_institucional
        if 'apoyo_institucional_ord' in df.columns:
            df['apoyo_institucional_ord'] = self._map_column(df['apoyo_institucional_ord'], self.mappings['apoyo_institucional_ord'])
        elif 'apoyo_institucional' in df.columns:
            df['apoyo_institucional_ord'] = self._map_column(df['apoyo_institucional'], self.mappings['apoyo_institucional_ord'])

        # satisfaccion_servicios
        if 'satisfaccion_servicios_ord' in df.columns:
            df['satisfaccion_servicios_ord'] = self._map_column(df['satisfaccion_servicios_ord'], self.mappings['satisfaccion_servicios_ord'])
        elif 'satisfaccion_servicios' in df.columns:
            df['satisfaccion_servicios_ord'] = self._map_column(df['satisfaccion_servicios'], self.mappings['satisfaccion_servicios_ord'])

        # actividades_extracurriculares
        if 'actividades_extracurriculares_ord' in df.columns:
            df['actividades_extracurriculares_ord'] = self._map_column(df['actividades_extracurriculares_ord'], self.mappings['actividades_extracurriculares_ord'])
        elif 'actividades_extracurriculares' in df.columns:
            df['actividades_extracurriculares_ord'] = self._map_column(df['actividades_extracurriculares'], self.mappings['actividades_extracurriculares_ord'])

        # EMPLEO / IMPACTO: one-hot using fixed categories (drop_first=True)
        # Empleo
        if 'empleo' in df.columns:
            df['empleo'] = pd.Categorical(df['empleo'], categories=self.empleo_cats)
        elif 'empleo_ord' in df.columns:
            # if numeric ordinal exists, try to map back to categories where possible
            # prefer textual column; if only numeric ordinal, create categories by ordinal values mapping
            inv = {0: "No", 1: "Sí, medio tiempo", 2: "Sí, tiempo completo"}
            df['empleo'] = df['empleo_ord'].map(inv).astype("category")
            df['empleo'] = pd.Categorical(df['empleo'], categories=self.empleo_cats)
        else:
            # ensure columns exist
            df['empleo'] = pd.Series(pd.Categorical([], categories=self.empleo_cats), index=df.index)

        empleo_dummies = pd.get_dummies(df['empleo'], prefix='empleo')
        # ensure consistent dummy columns names
        for cat in self.empleo_cats:
            col = f"empleo_{cat}"
            if col not in empleo_dummies.columns:
                empleo_dummies[col] = 0
        # drop first category to match JSON (reference "No")
        empleo_dummies = empleo_dummies.drop(columns=[f"empleo_{self.empleo_cats[0]}"])

        # Impacto laboral
        if 'impacto_laboral' in df.columns:
            df['impacto_laboral'] = pd.Categorical(df['impacto_laboral'], categories=self.impacto_cats)
        elif 'impacto_laboral_ord' in df.columns:
            inv_imp = {"No afecta / No trabajo": "No afecta / No trabajo", 0: "No afecta / No trabajo", 1: "Algo", 2: "Sí, mucho"}
            df['impacto_laboral'] = df['impacto_laboral_ord'].map(inv_imp).astype("category")
            df['impacto_laboral'] = pd.Categorical(df['impacto_laboral'], categories=self.impacto_cats)
        else:
            df['impacto_laboral'] = pd.Series(pd.Categorical([], categories=self.impacto_cats), index=df.index)

        impacto_dummies = pd.get_dummies(df['impacto_laboral'], prefix='impacto')
        for cat in self.impacto_cats:
            col = f"impacto_{cat}"
            if col not in impacto_dummies.columns:
                impacto_dummies[col] = 0
        impacto_dummies = impacto_dummies.drop(columns=[f"impacto_{self.impacto_cats[0]}"])

        # renombrar dummies a nombres más compactos (alineados a JSON idea)
        empleo_dummies = empleo_dummies.rename(columns={
            f"empleo_{self.empleo_cats[1]}": "empleo_Sí, medio tiempo",
            f"empleo_{self.empleo_cats[2]}": "empleo_Sí, tiempo completo"
        })
        impacto_dummies = impacto_dummies.rename(columns={
            f"impacto_{self.impacto_cats[1]}": "impacto_Algo",
            f"impacto_{self.impacto_cats[2]}": "impacto_Sí, mucho"
        })

        # componer dataframe final con orden fijo
        final_cols: List[str] = []
        # añadir numéricas (asegurar existencia)
        for c in self.features_numericas:
            if c not in df.columns:
                # si faltan columnas numéricas, crear con NaN (usuario puede imputar luego)
                df[c] = np.nan
            final_cols.append(c)

        # añadir one-hot en orden esperado
        for c in self.features_onehot:
            # map name to existing column in renamed dummies
            if c in empleo_dummies.columns:
                df[c] = empleo_dummies[c].astype(float)
            elif c in impacto_dummies.columns:
                df[c] = impacto_dummies[c].astype(float)
            else:
                # si no existe, agregar columna cero
                df[c] = 0.0
            final_cols.append(c)

        # resultado
        result = df[final_cols].copy()

        # Si hubiera NaN en columnas numéricas, dejarlas para que pipeline superior realice imputación si es necesario.
        return result

# Pipeline de ejemplo listo para usar en producción
def make_svm_pipeline():
    preprocessor = SVMPreprocessor()
    pipe = Pipeline([
        ("encoder", preprocessor),
        ("scaler", StandardScaler())
    ])
    return pipe

In [33]:
pipe = make_svm_pipeline()

pipe.fit(X_train)

/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the c

,steps,"[('encoder', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True


In [34]:
pipe.transform(X_train)

/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the c

array([[-1.37171857, -0.61324414,  1.79912259, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.90505143,  1.63067192,  1.79912259, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.90505143, -0.61324414, -0.55582649, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.23333357, -0.61324414, -0.55582649, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.90505143,  1.63067192, -0.55582649, ...,  0.        ,
         0.        ,  0.        ],
       [-1.37171857, -0.61324414,  1.79912259, ...,  0.        ,
         0.        ,  0.        ]], shape=(161, 10))

In [36]:
X_train.head(3)

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord
217,1er - 3er,Si,Sí,No,No,No afecta/No trabajo,Sí,Algunas veces,Insatisfecho/a,Nunca
104,7mo o más,Si,Sí,Sí,"Sí, medio tiempo",Algo,Sí,Algunas veces,Satisfecho/a,Algunas veces
19,7mo o más,No,Sí,No,"Sí, medio tiempo","Sí, mucho",No,Siempre,Muy satisfecho/a,Nunca


In [38]:
temp =pipe.transform(X_train)

/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return mapped.combine_first(fallback).astype(float)
/tmp/ipykernel_41245/3657229390.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the c

In [40]:
pd.DataFrame(temp, columns=list(X_train.columns))

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord
0,-1.371719,-0.613244,1.799123,-0.024329,-0.967849,-1.098012,0.0,0.0,0.0,0.0
1,0.905051,1.630672,1.799123,-0.024329,0.574960,0.231160,0.0,0.0,0.0,0.0
2,0.905051,-0.613244,-0.555826,1.934141,2.117768,-1.098012,0.0,0.0,0.0,0.0
3,0.905051,-0.613244,-0.555826,-1.982798,-0.967849,-1.098012,0.0,0.0,0.0,0.0
4,0.905051,-0.613244,-0.555826,1.934141,0.574960,0.231160,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
156,-0.233334,-0.613244,-0.555826,-0.024329,0.574960,0.231160,0.0,0.0,0.0,0.0
157,-1.371719,-0.613244,1.799123,-0.024329,0.574960,1.560332,0.0,0.0,0.0,0.0
158,-0.233334,-0.613244,-0.555826,1.934141,-2.510657,0.231160,0.0,0.0,0.0,0.0
159,0.905051,1.630672,-0.555826,-0.024329,0.574960,-1.098012,0.0,0.0,0.0,0.0
